In [1]:
## 设置数据库 & 定义数据模型
    
load './wtnb/wtnb.rb'

IRuby.display IRuby.html(include_js)
IRuby.display IRuby.html(include_css)
nil

"<script src='https://unpkg.com/vue/dist/vue.js'></script>\n<script src='https://unpkg.com/chart.js@2.9.3/dist/Chart.bundle.js'></script>\n<script src='https://unpkg.com/vue-chartkick@0.5.1/dist/vue-chartkick.js'></script>\n<script src='https://unpkg.com/element-ui/lib/index.js'></script>\n"

"<link rel='stylesheet' href='https://unpkg.com/element-ui/lib/theme-chalk/index.css'>\n"

In [7]:
def spu_history(spu,date)
    self.history = date
    
    product = ProductSpu.where(spu:spu).first
    
    if product.nil? then
        return [0,0,0,0,0]
        
    else

        #总库存
        total_stock = product.warehouse.size 

        #售出数量
        sold_stock = product.warehouse.sold.size

        #可用库存
        remain_stock = product.warehouse.unsold.size

        # 在单比
        in_order_rate = number_to_percentage(100- product.warehouse.inorder.size.to_f*100 / remain_stock )

        # 在途比
        stock = product.warehouse.unsold.includes(:item).map(&->(x){
            begin
                x.item.keep.last.orders.onroad
            rescue NoMethodError
                nil
            end
        })
        in_trip_rate = number_to_percentage(stock.count(true).to_f/stock.count*100)

        return [total_stock,sold_stock,remain_stock,in_order_rate,in_trip_rate]
    end
end
    
def spu_detail(spu,date_start,date_end)
    total_stock = {}
    sold_stock = {}
    remain_stock = {}
    in_order_rate = {}
    in_trip_rate = {}
    while (date_start < date_end) do
        date= date_start.to_s(:db)[0,10].gsub(/-/,"")
        d1,d2,d3,d4,d5  = spu_history(spu,date)      
        total_stock[date]=d1
        sold_stock[date]=d2
        remain_stock[date]=d3
        in_order_rate[date]=d4
        in_trip_rate[date]=d5
        
        date_start += 1.days
    end
    self.history = nil
    return [total_stock,sold_stock,remain_stock,in_order_rate,in_trip_rate]
end

product_list = []

[]

In [8]:
self.history=nil

feature_product = ProductSpuProperty.featured.product_spu.to_a
order_map = ProductSpuProperty.featured.product_spu.item.sold.group(:spu).count

feature_product.sort! { |x,y| -1 * ((order_map[x.spu] or 0)<=>(order_map[y.spu] or 0)) }

nil

In [ ]:
date_start = Time.zone.parse( (Time.now()-1.month).to_s(:db)[0,10])
date_end = Time.zone.parse( Time.now().to_s(:db)[0,10] )

feature_product.each do |x|
    spu = x.spu
    
    timebegin = Time.now()
    total_stock,sold_stock,remain_stock,in_order_rate,in_trip_rate = spu_detail(spu,date_start,date_end)
    timeend = Time.now()
    puts "Time: #{humanize(timeend-timebegin)}"

    product = {:name => spu.to_s + " " + ProductSpu.where(spu:spu).first.product_name,
          :picture => (ProductPicture.where(spu:spu).first_pic.map {|y| y.url }.first or ProductPicture.where(spu:spu).map {|y| y.url }.first), 
          :purchase_time => ProductSpu.where(spu:spu).first.warehouse.map do |x| x.create_time.to_s(:db)[0,10] end.uniq.size,
          :purchase_price => ProductSpu.where(spu:spu).first.purchase_price,
          :sold_amount => ProductSpu.where(spu:spu).first.item.orders.finish_purchase.item.keep.sold.where(spu:spu).map do |x| x.price_real.to_f end.inject(0){|sum,x| sum + x },
          :purchase_rate => ratio(ProductSpu.where(spu:spu).first.item.orders.finish_purchase.item.keep.where(spu:spu).sold.size , ProductSpu.where(spu:spu).first.item.orders.finish_purchase.item.keep.where(spu:spu).size),
          :total_stock => total_stock,
          :sold_stock => sold_stock,
          :remain_stock => remain_stock,
          :in_order_rate =>in_order_rate,
          :in_trip_rate =>in_trip_rate} 

    product_list.append([
     ratio( ratio(product[:sold_amount] , product[:sold_stock].to_a.last[1]),product[:purchase_price]),
     ratio(product[:sold_stock].to_a.last[1] ,product[:total_stock].to_a.last[1].to_f),
     ratio(product[:sold_amount],(product[:purchase_price]*product[:total_stock].to_a.last[1])),
     number_to_percentage(product[:purchase_rate]*100),
     product[:picture],
     product[:total_stock].to_a.last[1],
     product[:sold_stock].to_a.last[1],
     product[:remain_stock].to_a.last[1],
     product[:purchase_time],
     product[:purchase_price],
     product[:purchase_price]*product[:total_stock].to_a.last[1],
     ratio(product[:sold_amount] , product[:sold_stock].to_a.last[1]),
     product[:sold_amount].round,
     [{name:"总库存",data:product[:total_stock]}, {name:"售出数量",data:product[:sold_stock]}, {name:"可用库存",data:product[:remain_stock]}],
     [{name:"在单比",data:product[:in_order_rate]}, {name:"在途比",data:product[:in_trip_rate]}],
     product[:name],         
    ])

end

nil

Time: 16 seconds
Time: 16 seconds
Time: 23 seconds
Time: 16 seconds
Time: 8 seconds
Time: 12 seconds
Time: 13 seconds
Time: 15 seconds
Time: 15 seconds
Time: 15 seconds
Time: 16 seconds
Time: 17 seconds


In [5]:
puts_display render("主推品一览表",{:product_list=>product_list})
puts_to_file(render("主推品一览表",{:product_list=>product_list}),"output/主推品一览表.html")

"\n<div id=\"app_qNoAFvQCLE\">\n<el-container>\n\n  <el-container>\n\n  <el-header>\n    <el-row :gutter=\"24\">\n      <el-col :span=\"4\">\n        <el-button icon=\"el-icon-caret-left\" circle v-on:click=\"left\"></el-button>\n        <el-button icon=\"el-icon-caret-right\" circle v-on:click=\"right\"></el-button>\n      </el-col>\n\n      <el-col :span=\"20\">\n        <h1>[{{ cur_product+1 }}] {{ product_list[cur_product][15] }}</h1>\n      </el-col>\n    </el-row>\n  </el-header>\n  <el-main>\n    <el-divider></el-divider>\n    <el-row>\n      <el-col :span=\"12\">\n        <el-button type=\"primary\" round>加价倍数: {{ product_list[cur_product][0] }}</el-button>\n        <el-button type=\"primary\" round>售罄率: {{ product_list[cur_product][1] }}</el-button>\n        <el-button type=\"success\" round>回本率: {{ product_list[cur_product][2] }}</el-button>\n        <el-button type=\"success\" round>购买转化率: {{ product_list[cur_product][3] }}</el-button>\n      </el-col>\n    </el-row>\n    <el-row :gutter=\"20\">\n      <el-col :span=\"8\">\n        <img style=\"width: 100%;\" v-bind:src=\"product_list[cur_product][4]\">\n      </el-col>\n      <el-col :span=\"16\">\n        <line-chart :data=\"chart1\"></line-chart>\n        <line-chart :data=\"chart2\"></line-chart>\n        <el-row>\n          <el-button round>采购件数: {{ product_list[cur_product][5] }}</el-button>\n          <el-button round>销售件数: {{ product_list[cur_product][6] }}</el-button>\n          <el-button round>库存件数: {{ product_list[cur_product][7] }}</el-button>\n          <el-button round>补货次数: {{ product_list[cur_product][8] }}</el-button>\n          <el-button round>采购价格: {{ product_list[cur_product][9] }}</el-button>\n          <el-button round>采购成本: {{ product_list[cur_product][10] }}</el-button>\n          <el-button round>销售价格: {{ product_list[cur_product][11] }}</el-button>\n          <el-button round>销售总额: {{ product_list[cur_product][12] }}</el-button>\n        </el-row>\n      </el-col>\n    </el-row>\n    </el-main>\n  </el-container>\n</el-container>\n</div>\n\n\n\n<script>\n  Vue.use(VueChartkick.use(Chart))\n\n  app = new Vue({\n    el: '#app_qNoAFvQCLE',\n    data: {\n      product_list: [[\"3.7\",0.88,\"3.24\",\"35.000%\",\"https://retry-image.oss-cn-zhangjiakou.aliyuncs.com/product/spu/1efe183d-996c-4414-ae01-e154314aac9a.jpg\",105,92,12,10,\"46.0\",\"4830.0\",170.18,15657,[{\"name\":\"总库存\",\"data\":{\"20200412\":105}},{\"name\":\"售出数量\",\"data\":{\"20200412\":92}},{\"name\":\"可用库存\",\"data\":{\"20200412\":12}}],[{\"name\":\"在单比\",\"data\":{\"20200412\":\"75.000%\"}},{\"name\":\"在途比\",\"data\":{\"20200412\":\"0.000%\"}}],\"10341 博主同款极致显瘦牛仔长裤\"],[\"3.26\",0.7,\"2.27\",\"33.000%\",\"https://retry-image.oss-cn-zhangjiakou.aliyuncs.com/product/spu/a9e8f097-92e5-4934-9364-ce940d149f27.jpg\",43,30,13,6,\"52.0\",\"2236.0\",169.54,5086,[{\"name\":\"总库存\",\"data\":{\"20200412\":43}},{\"name\":\"售出数量\",\"data\":{\"20200412\":30}},{\"name\":\"可用库存\",\"data\":{\"20200412\":13}}],[{\"name\":\"在单比\",\"data\":{\"20200412\":\"92.308%\"}},{\"name\":\"在途比\",\"data\":{\"20200412\":\"15.385%\"}}],\"20655 高腰微喇显瘦牛仔裤\"],[\"3.31\",0.37,\"1.22\",\"19.000%\",\"https://retry-image.oss-cn-zhangjiakou.aliyuncs.com/product/spu/cd0ff93b-b9bc-4c78-98eb-b28ae6ebc5c1.jpg\",68,25,43,3,\"92.0\",\"6256.0\",304.33,7608,[{\"name\":\"总库存\",\"data\":{\"20200412\":68}},{\"name\":\"售出数量\",\"data\":{\"20200412\":25}},{\"name\":\"可用库存\",\"data\":{\"20200412\":43}}],[{\"name\":\"在单比\",\"data\":{\"20200412\":\"88.372%\"}},{\"name\":\"在途比\",\"data\":{\"20200412\":\"4.651%\"}}],\"20678 干练职业显瘦西装裤\"],[\"2.34\",0.48,\"1.11\",\"18.000%\",\"https://image.wetry.shop/product/spu/10294/10294_a.jpg\",42,20,20,7,\"79.0\",\"3318.0\",184.87,3697,[{\"name\":\"总库存\",\"data\":{\"20200412\":42}},{\"name\":\"售出数量\",\"data\":{\"20200412\":20}},{\"name\":\"可用库存\",\"data\":{\"20200412\":20}}],[{\"name\":\"在单比\",\"data\":{\"20200412\":\"85.000%\"}},{\"name\":\"在途比\",\"data\":{\"20200412\":\"15.000%\"}}],\"10294 设计感牛仔阔腿裤\"